<a href="https://colab.research.google.com/github/aditya26091999/genetic-algo-team-selection/blob/master/genetic_algorithm_teamselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies and Dataset retrieval

In [118]:
!pip install termcolor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from termcolor import colored
import random as rn

In [119]:
#loading the dataset from Github folder
datasetUrl = 'https://raw.githubusercontent.com/aditya26091999/genetic-algo-team-selection/master/SCOA_Dataset.csv?token=AMREUDFFUSQ4AQGENVAWS4DAXME7Q'
dataset = pd.read_csv(datasetUrl)

#checking the dataset loaded from Github
display(dataset.head())

,Timestamp,Name,Email,User Interface (UI) [UI],API (Server Side programming) [API],Languages [Language],Tools [Tools],Databases [DB],Cloud [Cloud],Artificial Intelligence [AI],Internet of Things [IOT],Testing [Testing],Overview of your skillset [SkillSet]
0,5/22/2021 10:53:05,Aditya Kawale,adityamkawale@gmail.com,"HTML, CSS, JavaScript, Angular, React, Bootstrap","Flask, Django, Anvil","Python, Java","Jenkins, Docker, Git, JIRA","MySQL, Oracle, MongoDB, SQLite","Heroku, Firebase, Azure",Speech Recognition,RaspberryPi,"JUnit, Selenium, Postman, SoapUI","FrontEnd, BackEnd, Databases, Testing, Cloud, ..."
1,5/22/2021 10:58:35,Chirag Kriplani,chiragkriplani@yahoo.in,"HTML, CSS, JavaScript, Angular, Native Android...",Spring,"Java, C++, C","Git, JIRA","MySQL, MongoDB",NaN,"Tensorflow, Keras, Computer Vision","RaspberryPi, Arduino, Motors",Postman,"FrontEnd, BackEnd, Databases"
2,5/22/2021 10:59:22,Hrishikesh Badakh,hrishikeshbadakh@gmail.com,"HTML, CSS, JavaScript, Angular, React, Bootstrap",Django,"Python, Java, C++","Kubernetes, Git, JIRA","MySQL, Oracle, MongoDB","Heroku, AWS","Tensorflow, Keras","RaspberryPi, Arduino, RFID module",NaN,FrontEnd
3,5/22/2021 11:01:29,Tanuj Balkhande,tbalkhande010@gmail.com,"HTML, CSS, JavaScript, Angular, Bootstrap",Flask,"Python, C++, C","Windows Shell Script, Linux Bash, Powershell, Git","MySQL, MongoDB",Heroku,NaN,NaN,NaN,FrontEnd
4,5/22/2021 11:01:30,Sushil Kamble,iamsushil303@gmail.com,"HTML, CSS, JavaScript, React, Bootstrap",Django,"Python, Java, C++","Windows Shell Script, Linux Bash, Powershell, Git","MySQL, Oracle, SQLServer, PostgresSQL, MongoDB...","Heroku, Firebase, GCP",NaN,NaN,NaN,"FrontEnd, BackEnd, Databases, Cloud, ProjectMa..."


In [120]:
print(colored("Original dataset columns\n","blue"))
print(dataset.columns)
print(colored("\nOriginal dataset shape : {}".format(str(dataset.shape)),"blue"))

dataset.drop(['Timestamp'],axis=1,inplace=True)
dataset.rename(columns={'User Interface (UI) [UI]': 'UI','API (Server Side programming) [API]':'API','Languages [Language]':'Language','Tools [Tools]':'Tools','Databases [DB]':'DB','Cloud [Cloud]':'Cloud','Artificial Intelligence [AI]':'AI','Internet of Things [IOT]':'IOT','Testing [Testing]':'Testing','Overview of your skillset [SkillSet]':'SkillSet'}, inplace=True)
print(colored("\nAfter removing obsolete columns\n","blue"))
print(dataset.columns)
print(colored("\nPreprocessed dataset shape : {}".format(str(dataset.shape)),"blue"))

dataset.head()

Original dataset columns

Index(['Timestamp', 'Name', 'Email', 'User Interface (UI) [UI]',
       'API (Server Side programming) [API]', 'Languages [Language]',
       'Tools [Tools]', 'Databases [DB]', 'Cloud [Cloud]',
       'Artificial Intelligence [AI]', 'Internet of Things [IOT]',
       'Testing [Testing]', 'Overview of your skillset [SkillSet]'],
      dtype='object')

Original dataset shape : (78, 13)

After removing obsolete columns

Index(['Name', 'Email', 'UI', 'API', 'Language', 'Tools', 'DB', 'Cloud', 'AI',
       'IOT', 'Testing', 'SkillSet'],
      dtype='object')

Preprocessed dataset shape : (78, 12)


,Name,Email,UI,API,Language,Tools,DB,Cloud,AI,IOT,Testing,SkillSet
0,Aditya Kawale,adityamkawale@gmail.com,"HTML, CSS, JavaScript, Angular, React, Bootstrap","Flask, Django, Anvil","Python, Java","Jenkins, Docker, Git, JIRA","MySQL, Oracle, MongoDB, SQLite","Heroku, Firebase, Azure",Speech Recognition,RaspberryPi,"JUnit, Selenium, Postman, SoapUI","FrontEnd, BackEnd, Databases, Testing, Cloud, ..."
1,Chirag Kriplani,chiragkriplani@yahoo.in,"HTML, CSS, JavaScript, Angular, Native Android...",Spring,"Java, C++, C","Git, JIRA","MySQL, MongoDB",NaN,"Tensorflow, Keras, Computer Vision","RaspberryPi, Arduino, Motors",Postman,"FrontEnd, BackEnd, Databases"
2,Hrishikesh Badakh,hrishikeshbadakh@gmail.com,"HTML, CSS, JavaScript, Angular, React, Bootstrap",Django,"Python, Java, C++","Kubernetes, Git, JIRA","MySQL, Oracle, MongoDB","Heroku, AWS","Tensorflow, Keras","RaspberryPi, Arduino, RFID module",NaN,FrontEnd
3,Tanuj Balkhande,tbalkhande010@gmail.com,"HTML, CSS, JavaScript, Angular, Bootstrap",Flask,"Python, C++, C","Windows Shell Script, Linux Bash, Powershell, Git","MySQL, MongoDB",Heroku,NaN,NaN,NaN,FrontEnd
4,Sushil Kamble,iamsushil303@gmail.com,"HTML, CSS, JavaScript, React, Bootstrap",Django,"Python, Java, C++","Windows Shell Script, Linux Bash, Powershell, Git","MySQL, Oracle, SQLServer, PostgresSQL, MongoDB...","Heroku, Firebase, GCP",NaN,NaN,NaN,"FrontEnd, BackEnd, Databases, Cloud, ProjectMa..."


# Genome Making - Approach 1
(Using SkillSet column in dataset)

In [ ]:
genome_columns = ['Name','Email','FrontEnd','BackEnd','Databases','Testing','Cloud','AI','IOT','ProjectManagement']
skill_set = ['FrontEnd','BackEnd','Databases','Testing','Cloud','AI','IOT','ProjectManagement']
genome_dataset = pd.DataFrame(columns=genome_columns)

for student in dataset.values:
  student_name = student[0]
  student_email= student[1]
  student_skillset = student[11].replace(" ","").split(',')
  temp_dict = dict()
  temp_dict['Name'] = student_name
  temp_dict['Email'] = student_email
  for i in skill_set:
    if i in student_skillset:
      temp_dict[i]=1
    else:
      temp_dict[i]=0
  genome_dataset = genome_dataset.append(temp_dict,ignore_index=True)

display(genome_dataset.head())

print(colored("\nShape of genome dataframe : {}".format(str(genome_dataset.shape)),"blue"))

display(genome_dataset.describe().T)


,Name,Email,FrontEnd,BackEnd,Databases,Testing,Cloud,AI,IOT,ProjectManagement
0,Aditya Kawale,adityamkawale@gmail.com,1,1,1,1,1,0,0,1
1,Chirag Kriplani,chiragkriplani@yahoo.in,1,1,1,0,0,0,0,0
2,Hrishikesh Badakh,hrishikeshbadakh@gmail.com,1,0,0,0,0,0,0,0
3,Tanuj Balkhande,tbalkhande010@gmail.com,1,0,0,0,0,0,0,0
4,Sushil Kamble,iamsushil303@gmail.com,1,1,1,0,1,0,0,1



Shape of genome dataframe : (78, 10)


,count,unique,top,freq
Name,78,78,Prathmesh Pethkar,1
Email,78,78,divekarsiddhi27@gmail.com,1
FrontEnd,78,2,1,60
BackEnd,78,2,1,50
Databases,78,2,1,57
Testing,78,2,0,69
Cloud,78,2,0,65
AI,78,2,0,70
IOT,78,2,0,71
ProjectManagement,78,2,0,59


# Genome Making - Approach 2
(Using percentage based skills)

Seeing count of skills per student

In [121]:
genome_columns = ['Name','Email','UI', 'API', 'Language', 'Tools', 'DB', 'Cloud', 'AI','IOT', 'Testing']
genome_dataset = pd.DataFrame(columns=genome_columns)

dataset.replace(np.nan, "",inplace=True)

dataset_copy = dataset.copy(deep=True)

for j in dataset_copy.columns[2:-1]:
  for i,val in enumerate(dataset_copy[j]):
      dataset_copy[j][i] = len(val.replace(" ","").split(','))

display(dataset_copy.head())

,Name,Email,UI,API,Language,Tools,DB,Cloud,AI,IOT,Testing,SkillSet
0,Aditya Kawale,adityamkawale@gmail.com,6,3,2,4,4,3,1,1,4,"FrontEnd, BackEnd, Databases, Testing, Cloud, ..."
1,Chirag Kriplani,chiragkriplani@yahoo.in,6,1,3,2,2,1,3,3,1,"FrontEnd, BackEnd, Databases"
2,Hrishikesh Badakh,hrishikeshbadakh@gmail.com,6,1,3,3,3,2,2,3,1,FrontEnd
3,Tanuj Balkhande,tbalkhande010@gmail.com,5,1,3,4,2,1,1,1,1,FrontEnd
4,Sushil Kamble,iamsushil303@gmail.com,5,1,3,4,6,3,1,1,1,"FrontEnd, BackEnd, Databases, Cloud, ProjectMa..."


Seeing percentage share of each skill

In [122]:
genome_columns = ['Name','Email','UI', 'API', 'Language', 'Tools', 'DB', 'Cloud', 'AI','IOT', 'Testing']
genome_dataset = pd.DataFrame(columns=genome_columns)

dataset.replace(np.nan, "",inplace=True)

for j in dataset.columns[2:-1]:
  for i,val in enumerate(dataset[j]):
      dataset[j][i] = len(val.replace(" ","").split(','))/8

display(dataset.head())

,Name,Email,UI,API,Language,Tools,DB,Cloud,AI,IOT,Testing,SkillSet
0,Aditya Kawale,adityamkawale@gmail.com,0.75,0.375,0.25,0.5,0.5,0.375,0.125,0.125,0.5,"FrontEnd, BackEnd, Databases, Testing, Cloud, ..."
1,Chirag Kriplani,chiragkriplani@yahoo.in,0.75,0.125,0.375,0.25,0.25,0.125,0.375,0.375,0.125,"FrontEnd, BackEnd, Databases"
2,Hrishikesh Badakh,hrishikeshbadakh@gmail.com,0.75,0.125,0.375,0.375,0.375,0.25,0.25,0.375,0.125,FrontEnd
3,Tanuj Balkhande,tbalkhande010@gmail.com,0.625,0.125,0.375,0.5,0.25,0.125,0.125,0.125,0.125,FrontEnd
4,Sushil Kamble,iamsushil303@gmail.com,0.625,0.125,0.375,0.5,0.75,0.375,0.125,0.125,0.125,"FrontEnd, BackEnd, Databases, Cloud, ProjectMa..."


Now we have values for fitness function, now we will start with Genetic Algorithm

# Creating genetic algorithm classes

Creating Population for genetic algorithm

In [309]:
class Population:

  #Initialising Population class
  def __init__(self, populationSize,total_students,teamSize):
    self.populationSize = populationSize
    self.total_students = total_students
    self.teamSize = teamSize
    self.population = [] #genomes for GA
    self.random_selected_students = [] #students ids for fitness
    self.students = pd.DataFrame()
  
  #Printing details of Population object
  def __str__(self):
    return ("Initial Population has {} genomes\nEach genome has {} students.\nEach genome has only {} team members".format(self.populationSize, self.total_students,self.teamSize))

  #Making initial population of popultionSize count
  def initGenomes(self):
    for j in range(1,self.populationSize+1):
      i = Individual(genomeSize = self.total_students,teamSize=self.teamSize)
      i.selectTeamMembers()
      print("Team #{}\n{}".format(j,i),end="\n\n")
      self.population.append(i)

  #Bring total_students details for GA from global dataset(Github/Cleaned)
  def createStudentDataset(self):
    self.random_selected_students = rn.sample(range(0,dataset.shape[0]-1),self.total_students)
    print(self.random_selected_students)

  def showCreatedStudents(self):
      self.students = dataset.filter(self.random_selected_students,axis=0)
      display(self.students)
    
  #calculates the fitness for each team
  def calc_fitness(self,genome,project):

    fitness_score = 0

    skills = list(project['skills'].keys())
    skills_val = list(project['skills'].values())

    total_skill_of_team = [0]*len(skills)
    avg_skill_of_team = [0]*len(skills)
    team_member_count = 0

    for i,val in enumerate(genome):
      if val==1:
        team_member_count+=1
        student_index = self.random_selected_students[i]
        for j,skill in enumerate(skills):
          total_skill_of_team[j] += self.students.loc[student_index,:][skill]

    for k,val in enumerate(total_skill_of_team):
      avg_skill_of_team[k] = val/self.teamSize

    #Check for fitness of genome


    #Criteria 1
    if(team_member_count!=self.teamSize):
      fitness_score-=100 
    else:
      fitness_score+=1 #Increased point
    
    #Criteria 2
    for i,val in enumerate(skills_val):
      if(avg_skill_of_team[i] >= val):
        fitness_score+=1 #Increased point
      else:
        pass

    return fitness_score

  #Should return top 2 teams
  def checkFittest(self,project):
    for i in self.population:
      print(self.calc_fitness(i.genome,project))
    pass
  


In [310]:
class Individual:
  def __init__(self,genomeSize,teamSize):
    self.genomeSize = genomeSize
    self.teamSize = teamSize
    self.genome = [0]*genomeSize

  def __str__(self):
    return ("Individual has {} students.\nOut of these {} students, only {} are considered in team.\nThe team members are at position: {}\nTeam looks like : {}\n".format(self.genomeSize,self.genomeSize,self.teamSize,self.teamMemberIndexList,self.genome))
 
  def selectTeamMembers(self):
    self.teamMemberIndexList = rn.sample(range(0,self.genomeSize-1),self.teamSize)
    for i in self.teamMemberIndexList:
      self.genome[i] = 1

In [311]:
p = Population(populationSize=8, total_students=10, teamSize=4)
print(p,end="\n\n")
print(colored("These are teams\n","blue"))
p.initGenomes()

Initial Population has 8 genomes
Each genome has 10 students.
Each genome has only 4 team members

These are teams

Team #1
Individual has 10 students.
Out of these 10 students, only 4 are considered in team.
The team members are at position: [1, 7, 3, 0]
Team looks like : [1, 1, 0, 1, 0, 0, 0, 1, 0, 0]


Team #2
Individual has 10 students.
Out of these 10 students, only 4 are considered in team.
The team members are at position: [1, 3, 5, 7]
Team looks like : [0, 1, 0, 1, 0, 1, 0, 1, 0, 0]


Team #3
Individual has 10 students.
Out of these 10 students, only 4 are considered in team.
The team members are at position: [6, 2, 8, 4]
Team looks like : [0, 0, 1, 0, 1, 0, 1, 0, 1, 0]


Team #4
Individual has 10 students.
Out of these 10 students, only 4 are considered in team.
The team members are at position: [0, 1, 6, 8]
Team looks like : [1, 1, 0, 0, 0, 0, 1, 0, 1, 0]


Team #5
Individual has 10 students.
Out of these 10 students, only 4 are considered in team.
The team members are at pos

In [312]:
p.createStudentDataset()
p.showCreatedStudents()

[2, 33, 73, 41, 24, 68, 50, 71, 19, 13]


,Name,Email,UI,API,Language,Tools,DB,Cloud,AI,IOT,Testing,SkillSet
2,Hrishikesh Badakh,hrishikeshbadakh@gmail.com,0.75,0.125,0.375,0.375,0.375,0.25,0.25,0.375,0.125,FrontEnd
33,Atharv Deshmukh,hi.atharv.deshmukh@gmail.com,0.625,0.25,0.375,0.25,0.25,0.25,0.125,0.625,0.125,"FrontEnd, BackEnd, Databases"
73,Snehal kuldhar,pranjalimalunjkar1@gmail.com,0.125,0.125,0.25,0.125,0.125,0.125,0.125,0.125,0.125,FrontEnd
41,Siddhi Divekar,divekarsiddhi27@gmail.com,0.25,0.125,0.25,0.125,0.25,0.125,0.125,0.125,0.125,"FrontEnd, Databases, ProjectManagement"
24,Chirag Bagde,chiragsbagde@gmail.com,0.625,0.125,0.25,0.375,0.125,0.125,0.125,0.125,0.125,"FrontEnd, Databases"
68,Ashutosh Agrawal,ashutosh.17u580@viit.ac.in,0.625,0.125,0.25,0.5,0.25,0.125,0.375,0.125,0.25,"FrontEnd, BackEnd"
50,Vipul Chaudhari,vipulchaudhary2005@gmail.com,0.375,0.375,0.375,0.375,0.375,0.125,0.5,0.5,0.25,"FrontEnd, BackEnd, Databases, ProjectManagement"
71,Pratik Gawali,pratikgawali06@gmail.com,0.25,0.125,0.25,0.125,0.25,0.125,0.125,0.125,0.125,"FrontEnd, Databases"
19,Vishal Dusane,vishalrd1999@gmail.com,0.5,0.25,0.375,0.25,0.375,0.125,0.125,0.125,0.25,"BackEnd, Databases"
13,Tejaswini Yesugade,tejaswini30699@gmail.com,0.625,0.25,0.5,0.375,0.375,0.125,0.375,0.125,0.375,"FrontEnd, BackEnd, Databases"


In [313]:
print(p.calc_fitness([1, 1, 1, 1, 0, 0, 0, 0, 0, 0],projects[0]))
print(p.calc_fitness([1, 1, 1, 0, 0, 0, 0, 0, 0, 0],projects[0]))


3
-99


Creating Individual for population

Dummy data for testing GA

In [314]:
projects = list()
projects.append({'project_number': 1, 'skills': {'Cloud': 0.4, 'IOT': 0.4, 'DB': 0.1,'Testing':0.1}})
projects.append({'project_number': 2, 'skills': {'UI':0.5,'API':0.3,'DB':0.2}})
projects.append({'project_number': 3, 'skills':{'UI':0.3,'AI':0.6,'Tools':0.1}})
print(projects)

[{'project_number': 1, 'skills': {'Cloud': 0.4, 'IOT': 0.4, 'DB': 0.1, 'Testing': 0.1}}, {'project_number': 2, 'skills': {'UI': 0.5, 'API': 0.3, 'DB': 0.2}}, {'project_number': 3, 'skills': {'UI': 0.3, 'AI': 0.6, 'Tools': 0.1}}]


In [315]:
p.checkFittest(projects[0])

3
3
3
4
3
3
3
3
